## MultiModal PDF-RAG with Langchain and Milvus Vector database

 ### Load and parse Multimodal PDF

In [ ]:
import os

#os.environ["PATH"] += os.pathsep + "/opt/homebrew/bin"
from unstructured.partition.pdf import partition_pdf 

raw_pdf_elements=partition_pdf(
    filename="data/SIABestPractices.pdf", 
    strategy="hi_res",                                      
    extract_images_in_pdf=True,                            
    extract_image_block_output_dir="extracted_data",
    infer_table_structure=True,  
    )


Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
I0000 00:00:1749225890.126286 3969714 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
I0000 00:00:1749225890.164007 3969714 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
I0000 00:00:17

In [186]:
raw_pdf_elements

In [ ]:
raw_pdf_elements
Title=[]
Header=[]
Footer=[]
Image=[]
NarrativeText=[]
Text=[]
ListItem=[]
Table=[]

for element in raw_pdf_elements:
    if "unstructured.documents.elements.Title" in str(type(element)):
        Title.append(str(element))
    elif "unstructured.documents.elements.Header" in str(type(element)):
        Header.append(str(element))
    elif "unstructured.documents.elements.Footer" in str(type(element)):
        Footer.append(str(element))

    elif "unstructured.documents.elements.Image" in str(type(element)):
        Image.append(str(element))
    elif "unstructured.documents.elements.NarrativeText" in str(type(element)):
        NarrativeText.append(str(element))
    elif "unstructured.documents.elements.Text" in str(type(element)):
        Text.append(str(element))
    elif "unstructured.documents.elements.ListItem" in str(type(element)):
        ListItem.append(str(element))
    elif "unstructured.documents.elements.Table" in str(type(element)):
        Table.append(str(element))
    

In [158]:
NarrativeText


['Search',
 'Secure Infrastructure Access (SIA) offers an agentless, SaaS solution for securing privileged access to organizational assets across hybrid and cloud environments. It supports access with zero standing privileges (ZSP) or using vaulted credentials, reducing the risk of standing privileges and including session isolation and monitoring. This document offers the current technical best practices for deploying and conﬁguring SIA securely, architecture for SIA connectors, general conﬁguration of SIA, and the operational aspects associated with maintaining a functional deployment.',
 '21-Oct-2024 Knowledge Article',
 'Security Best Practices for Hardening Your CyberArk Identity Deployment (/s/article/Security-Best- Practices-for-Hardening-Your-CyberArk-Identity- Deployment) 1.48K',
 'SIA Best Practices: Architecture, Conﬁguration, Security, and Operations',
 'Workforce Password Manager Best Practices (/s/article/Workforce-Password-Manager-Best- Practices) 404',
 'Jake DeSantis, 

In [159]:
Text

['1,962', '21 October 2024 at 11:52', '=', '21/10/2024 11:52']

In [160]:
ListItem

['(/s/proﬁle/005Ht00000AibqgIAB) (CyberArk) published a new version of this Knowledge Article.',
 'LiadD (CyberArk Community Manager) (/s/proﬁle/005Ht00000AibqgIAB) (CyberArk) unpublished this Knowledge Article. 21 October 2024 at 11:52 (/s/feed/0D5Ht0000AUrPf7KQF)',
 'Sal (CyberArk Community Manager)',
 '(/s/proﬁle/0052J000009GKbpQAG) (CyberArk) published a new version of this Knowledge Article. 14 October 2024 at 15:37 (/s/feed/0D5Ht0000AUPHxIKQX)',
 'Like Comment',
 'Sal (CyberArk Community Manager)',
 '(/s/proﬁle/0052J000009GKbpQAG) (CyberArk) published a new version of this Knowledge Article. 14 October 2024 at 15:34 (/s/feed/0D5Ht0000AUPHu0KQH)',
 'James Creamer (CyberArk)',
 '(/s/proﬁle/005500000062PIFAA2) (CyberArk) published a new version of this Knowledge Article. 17 July 2024 at 20:13 (/s/feed/0D5Ht0000AQRLyMKQX)',
 'Sal (CyberArk Community Manager)',
 '(/s/proﬁle/0052J000009GKbpQAG) (CyberArk) published a new version of this Knowledge Article. 15 July 2024 at 16:01 (/s/feed

In [161]:
Image[2]

'@ COOKIE PREFERENCES 7'

In [162]:
Table[0]

'On-premises assets operator Hostname/FODN, Domainysuttic'

### Convert Elements into Langchain Documents

In [163]:
#Convert Elements into Langchain Documents

from langchain.schema import Document
documents =[]
for ele in raw_pdf_elements:
    if ele.text:
        documents.append(
            Document(
            page_content=ele.text,
            metadata={"type": ele.category, "page_number": ele.metadata.page_number}
            )
            )
    elif ele.category =="image" and ele.metadata== "image_path":
        documents.append(Document(
            page_content=f"[Image: {ele.metadata.image_path}]",
            metadata={"type": "image", "page_number":ele.metadata.page_number}
        ))

In [164]:
documents

[Document(metadata={'type': 'Title', 'page_number': 1}, page_content='Home (/s/)'),
 Document(metadata={'type': 'Header', 'page_number': 1}, page_content='Engage Updates Product Centers Resources'),
 Document(metadata={'type': 'Title', 'page_number': 1}, page_content='(Pore'),
 Document(metadata={'type': 'Title', 'page_number': 1}, page_content='More'),
 Document(metadata={'type': 'Title', 'page_number': 1}, page_content='Have a Question? Ask the Community'),
 Document(metadata={'type': 'Title', 'page_number': 1}, page_content='(Q Search...'),
 Document(metadata={'type': 'Title', 'page_number': 1}, page_content='Search...'),
 Document(metadata={'type': 'Title', 'page_number': 1}, page_content='Search )'),
 Document(metadata={'type': 'NarrativeText', 'page_number': 1}, page_content='Search'),
 Document(metadata={'type': 'Title', 'page_number': 1}, page_content='SIA Best Practices: Architecture, Conﬁguration, Security, and Operations'),
 Document(metadata={'type': 'NarrativeText', 'page_

### Embedd Documents

In [165]:
# Embedd documents
import os
os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")

from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


### Store in Milvus

In [ ]:
# Store in Milvus
# Milvus Lite 
### The easiest way to prototype is to use Milvus Lite, where everything is stored in a local vector database file. Only the Flat index can be used.

from langchain_milvus import Milvus

URI = "./milvus_example02.db"

vector_store = Milvus(
    embedding_function=embeddings,
    connection_args={"uri": URI},
    index_params={"index_type": "FLAT", "metric_type": "L2"},
)

I0000 00:00:1749226014.313046 3969714 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [168]:

from uuid import uuid4
uuids=[]
for i in range(len(documents)):
    uuids.append(str(uuid4()))

uuids


['c4546a4d-0ad2-4944-a247-48de337beb4b',
 'b25424dd-60dd-4704-874d-e1ff361ddfa8',
 '8620b389-2ac5-40eb-8cb7-b0a62ce06dc6',
 '5a4c3448-757c-4a9b-a2f5-a2fcc0a17b67',
 'dcc8064d-94f3-4965-a1c5-a938b120980c',
 '8a76435d-fee0-403d-a4e2-3ded9d518ce8',
 'f9381722-cd23-4332-b795-ff744c0e0ee0',
 '05982f2a-a27b-48e0-b357-cf8589642e56',
 'e0f6ad24-bcb9-44b2-be27-8430fbc6866c',
 '013c85d9-5ad5-4b3c-b6d3-54ddd2b0c074',
 'e17b4c4a-31a0-4f04-bb45-a26cd33b2aec',
 '768b2fbb-7e0c-4ee7-9a52-b190ebef96e3',
 '3701866f-01cf-44aa-9fef-81dc6f24bfb6',
 '692a07f2-36df-4276-b501-424277350b27',
 'e6761131-d082-473b-a984-d78a5ce1579b',
 '4494631c-fd3e-4cd8-9706-add5dce962d5',
 '41ca7555-1967-4130-9fa2-8637fd051587',
 'f2055834-d36d-4367-ada2-12829be7e2e8',
 '5fe6d303-a79d-437e-a09d-0386632a1a22',
 '6462e215-e024-42bb-8baa-352e6873bf19',
 '7ce9c62c-96a7-4ec1-b547-8614db13e576',
 'fc4fb1aa-212a-457b-a82a-7661183235fa',
 'dfce3421-2207-42b7-82f6-9ea73e1a5ecd',
 'cb1d5f14-e5f7-4b75-8b7d-9cb884118f11',
 '498f92f7-ee44-

### Add items to vector store

In [169]:
#Add items to vector store
vector_store.add_documents(documents=documents, ids=uuids)

['c4546a4d-0ad2-4944-a247-48de337beb4b',
 'b25424dd-60dd-4704-874d-e1ff361ddfa8',
 '8620b389-2ac5-40eb-8cb7-b0a62ce06dc6',
 '5a4c3448-757c-4a9b-a2f5-a2fcc0a17b67',
 'dcc8064d-94f3-4965-a1c5-a938b120980c',
 '8a76435d-fee0-403d-a4e2-3ded9d518ce8',
 'f9381722-cd23-4332-b795-ff744c0e0ee0',
 '05982f2a-a27b-48e0-b357-cf8589642e56',
 'e0f6ad24-bcb9-44b2-be27-8430fbc6866c',
 '013c85d9-5ad5-4b3c-b6d3-54ddd2b0c074',
 'e17b4c4a-31a0-4f04-bb45-a26cd33b2aec',
 '768b2fbb-7e0c-4ee7-9a52-b190ebef96e3',
 '3701866f-01cf-44aa-9fef-81dc6f24bfb6',
 '692a07f2-36df-4276-b501-424277350b27',
 'e6761131-d082-473b-a984-d78a5ce1579b',
 '4494631c-fd3e-4cd8-9706-add5dce962d5',
 '41ca7555-1967-4130-9fa2-8637fd051587',
 'f2055834-d36d-4367-ada2-12829be7e2e8',
 '5fe6d303-a79d-437e-a09d-0386632a1a22',
 '6462e215-e024-42bb-8baa-352e6873bf19',
 '7ce9c62c-96a7-4ec1-b547-8614db13e576',
 'fc4fb1aa-212a-457b-a82a-7661183235fa',
 'dfce3421-2207-42b7-82f6-9ea73e1a5ecd',
 'cb1d5f14-e5f7-4b75-8b7d-9cb884118f11',
 '498f92f7-ee44-

### Creation of retriever class for vector store

In [170]:
#creation of retriever class for vector store

retriever =vector_store.as_retriever(
        search_type="similarity_score_threshold",
        search_kwargs={'score_threshold': 0.6,'k': 6} #hyperparameter
)

### Querying the VectorStore

In [171]:
# Query directly - Similarity search

res1= vector_store.similarity_search(query="what is strong accounts",k=3)
res1
for res in res1:
    print(f"# {res.page_content}")

# Strong Accounts
# Strong accounts are privileged accounts with elevated access rights that provide temporary accounts used for ZSP access. Strong accounts can be considered “Service Accounts” in that they are used by the SIA service to automate access for the ZSP ﬂow. To provision ephemeral users, SIA needs to access a strong account, which can create the ephemeral users on the target machines and delete them when the session ends.
# Strong accounts can either be speciﬁed in SIA directly or you can conﬁgure SIA to retrieve the account from the vault. Customers are advised to opt for vaulting and rotating the strong accounts. By vaulting strong accounts, Organizations can comply with business security objective and mitigate malicious or risky account usage. Additionally, regular rotation of strong account passwords helps to mitigate the risk of credential theft or misuse by malicious actors. This can be integrated with self-hosted deployments of CyberArk Privileged Access Management (

In [172]:
# Query directly - Similarity search with score

res2= vector_store.similarity_search_with_score(query="what is strong accounts",k=3)
res2


[(Document(metadata={'page_number': 3, 'pk': 'f78b398b-5cc4-445e-9310-25d1142a8388', 'type': 'Title'}, page_content='Strong Accounts'),
  0.3603437840938568),
 (Document(metadata={'page_number': 3, 'pk': '8afde83b-8751-45be-bc71-e60312e26a23', 'type': 'NarrativeText'}, page_content='Strong accounts are privileged accounts with elevated access rights that provide temporary accounts used for ZSP access. Strong accounts can be considered “Service Accounts” in that they are used by the SIA service to automate access for the ZSP ﬂow. To provision ephemeral users, SIA needs to access a strong account, which can create the ephemeral users on the target machines and delete them when the session ends.'),
  0.6745815277099609),
 (Document(metadata={'page_number': 3, 'pk': 'bd3f9b3a-89b4-4f2b-906e-d4a5ef525523', 'type': 'NarrativeText'}, page_content='Strong accounts can either be speciﬁed in SIA directly or you can conﬁgure SIA to retrieve the account from the vault. Customers are advised to opt

In [173]:
for res,score in res2:
    print(f"* {res.page_content},{score:3f}")
   

* Strong Accounts,0.360344
* Strong accounts are privileged accounts with elevated access rights that provide temporary accounts used for ZSP access. Strong accounts can be considered “Service Accounts” in that they are used by the SIA service to automate access for the ZSP ﬂow. To provision ephemeral users, SIA needs to access a strong account, which can create the ephemeral users on the target machines and delete them when the session ends.,0.674582
* Strong accounts can either be speciﬁed in SIA directly or you can conﬁgure SIA to retrieve the account from the vault. Customers are advised to opt for vaulting and rotating the strong accounts. By vaulting strong accounts, Organizations can comply with business security objective and mitigate malicious or risky account usage. Additionally, regular rotation of strong account passwords helps to mitigate the risk of credential theft or misuse by malicious actors. This can be integrated with self-hosted deployments of CyberArk Privileged A

In [174]:
# search_type = ""similarity"" 
# Returns the documents most similar to the query using a distance metric (like cosine similarity or Euclidean distance) between embedding vectors.

retriever =vector_store.as_retriever(
        #search_type="similarity_score_threshold",
        search_type="similarity",
        search_kwargs={'k': 6} #hyperparameter
)
response_1 =retriever.invoke("what is strong accounts?")

for doc in response_1:
    print(doc.page_content)


Strong Accounts
Strong accounts are privileged accounts with elevated access rights that provide temporary accounts used for ZSP access. Strong accounts can be considered “Service Accounts” in that they are used by the SIA service to automate access for the ZSP ﬂow. To provision ephemeral users, SIA needs to access a strong account, which can create the ephemeral users on the target machines and delete them when the session ends.
Strong accounts can either be speciﬁed in SIA directly or you can conﬁgure SIA to retrieve the account from the vault. Customers are advised to opt for vaulting and rotating the strong accounts. By vaulting strong accounts, Organizations can comply with business security objective and mitigate malicious or risky account usage. Additionally, regular rotation of strong account passwords helps to mitigate the risk of credential theft or misuse by malicious actors. This can be integrated with self-hosted deployments of CyberArk Privileged Access Management (PAM) b

In [175]:
# search_type = ""similarity_score_threshold"" 
# Only returns documents with a similarity score above a certain threshold.

retriever =vector_store.as_retriever(
        search_type="similarity_score_threshold",
        search_kwargs={"score_threshold": 0.6} #hyperparameter
)
response_2 =retriever.invoke("what is strong accounts?")

for doc in response_2:
    print(doc.page_content)

Strong Accounts
Strong accounts are privileged accounts with elevated access rights that provide temporary accounts used for ZSP access. Strong accounts can be considered “Service Accounts” in that they are used by the SIA service to automate access for the ZSP ﬂow. To provision ephemeral users, SIA needs to access a strong account, which can create the ephemeral users on the target machines and delete them when the session ends.
Strong accounts can either be speciﬁed in SIA directly or you can conﬁgure SIA to retrieve the account from the vault. Customers are advised to opt for vaulting and rotating the strong accounts. By vaulting strong accounts, Organizations can comply with business security objective and mitigate malicious or risky account usage. Additionally, regular rotation of strong account passwords helps to mitigate the risk of credential theft or misuse by malicious actors. This can be integrated with self-hosted deployments of CyberArk Privileged Access Management (PAM) b

In [176]:
# search_type = "mmr" 
# Maximal Marginal Relevance 
# # It tries to avoid redundancy among results.
retriever =vector_store.as_retriever(
        search_type="mmr",
        search_kwargs={'k': 6} #hyperparameter
)
res3 =retriever.invoke("what is strong accounts?")
res3
for doc in res3:
    print(doc.page_content)

Strong Accounts
Strong accounts are privileged accounts with elevated access rights that provide temporary accounts used for ZSP access. Strong accounts can be considered “Service Accounts” in that they are used by the SIA service to automate access for the ZSP ﬂow. To provision ephemeral users, SIA needs to access a strong account, which can create the ephemeral users on the target machines and delete them when the session ends.
While the strong accountant may be a member of the Domain Administrators group, this doesn't follow the principle of least privilege and is not considered best practice. To adhere to the principle of least privilege and separation of duties, organizations should subdivide strong accounts into privilege silos. This granular approach helps to ensure that the strong account does not have widespread access throughout your environment. You may also choose to select SIA strong accounts based on credential boundary or network area which may also help align access rig

### # Reranking
After the first stage of retrieval, we need to rerank the candidates to get a better result.

In [177]:
# Reranking
from langchain_openai import ChatOpenAI
from langchain.retrievers.document_compressors import LLMListwiseRerank

llm = ChatOpenAI(model="gpt-4o")
reranker = LLMListwiseRerank.from_llm(llm=llm)

# Apply reranking
reranked_docs = reranker.compress_documents(documents=res3, query="what is strong account?")

# Show results
for doc in reranked_docs:
    print(doc.page_content)

Strong accounts are privileged accounts with elevated access rights that provide temporary accounts used for ZSP access. Strong accounts can be considered “Service Accounts” in that they are used by the SIA service to automate access for the ZSP ﬂow. To provision ephemeral users, SIA needs to access a strong account, which can create the ephemeral users on the target machines and delete them when the session ends.
While the strong accountant may be a member of the Domain Administrators group, this doesn't follow the principle of least privilege and is not considered best practice. To adhere to the principle of least privilege and separation of duties, organizations should subdivide strong accounts into privilege silos. This granular approach helps to ensure that the strong account does not have widespread access throughout your environment. You may also choose to select SIA strong accounts based on credential boundary or network area which may also help align access rights with organiz

### RAG Pipeline

In [178]:
# RAG part

# LLM
from langchain_groq import ChatGroq
llm = ChatGroq(model ="llama-3.3-70b-versatile")

In [179]:
# prompt
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

In [180]:
# checking the prompt template variables...
import pprint
pprint.pprint(prompt.messages)

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


In [181]:
# output parser
from langchain_core.output_parsers import StrOutputParser

In [182]:
# chain creation
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

chain =(
    { 
      "context":retriever| RunnableLambda(format_docs),
      "question":RunnablePassthrough()
     }
    |prompt
    |llm
    |StrOutputParser()
)

In [183]:
response=chain.invoke("Explain the monitoring part of SIA operation?")
print(response)

The monitoring part of SIA operation involves tracking key metrics such as CPU usage, memory utilization, and network throughput. Comprehensive logging is also implemented to capture detailed information about SIA operations, including connection attempts and failures. Regular review of these logs helps identify patterns or anomalies that could indicate potential issues, with automated monitoring solutions providing real-time visibility into the SIA service's status and performance.


In [185]:
response=chain.invoke("list me the related articles for Hardening??")
print(response)


The related article for Hardening is "Security Best Practices for Hardening Your CyberArk Identity Deployment". This article provides guidance on securing and optimizing CyberArk Identity deployments. There is no list of multiple related articles provided in the given context.
